In [1]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Necessary imports

In [2]:
import numpy as np
import pandas as pd
import os
import sys
from tqdm import tqdm_notebook
import tensorflow as tf
import tensorflow_hub as hub
from nltk import sent_tokenize
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import math
import keras
import nltk
%matplotlib inline

Using TensorFlow backend.


In [0]:
# Loading the dataset from google drive
df = pd.read_csv("/content/drive/Shared drives/ChiragSaini SharedDrive/Precily Assignment/Text_Similarity_Dataset.csv")

In [4]:
df.head()

,Unique_ID,text1,text2
0,0,savvy searchers fail to spot ads internet sear...,newcastle 2-1 bolton kieron dyer smashed home ...
1,1,millions to miss out on the net by 2025 40% o...,nasdaq planning $100m share sale the owner of ...
2,2,young debut cut short by ginepri fifteen-year-...,ruddock backs yapp s credentials wales coach m...
3,3,diageo to buy us wine firm diageo the world s...,mci shares climb on takeover bid shares in us ...
4,4,be careful how you code a new european directi...,media gadgets get moving pocket-sized devices ...


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4023 entries, 0 to 4022
Data columns (total 3 columns):
Unique_ID    4023 non-null int64
text1        4023 non-null object
text2        4023 non-null object
dtypes: int64(1), object(2)
memory usage: 94.4+ KB


So in our Dataset, there are 4023 observation i.e. 4023 pairs of paragraphs

Preprocessing Data:
    Cleaning text and removing unwanted words and symbols

In [0]:
import re
def clean_paragraph(text_list):
    clean_list = []
    for sent in text_list:
        sent = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-.:;<=>?@[\]^`{|}~"""), '',sent)
        sent = sent.replace('[]','')
        sent = re.sub('\d+',' ',sent)
        sent = sent.lower()
        clean_list.append(sent)
    return clean_list

Embedding using Google universal encoder


In [6]:
# In this problem, we cant use word encodings.
# we need to use Sentence encoding as many sentences are similar in textual context,
# but not in semantic or logical context
# for example: "Lion killed Deer" and "Deer killer Lion" are similar in textual context but different in logical context
nltk.download("punkt")
X=[]
with tf.Graph().as_default():
    #loading model
    embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/2")
    print("embed",embed)
    messages = tf.placeholder(dtype=tf.string, shape=[None])
    output = embed(messages)
    #tensorflow session
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        for text in ["text1","text2"]:
            phrases_emb_list = []
            for phrases in tqdm_notebook(df[text]):
                #breaking paragraph into sentences  
                sent_list = sent_tokenize(phrases)
                #cleaning the sentence
                clean_sent_list = clean_paragraph(sent_list)
                #feeding sentence in the encoder
                sent_embed = session.run(output, feed_dict={messages: clean_sent_list})
                #appending the encodings in a list
                phrases_emb_list.append(sent_embed.mean(axis=0).reshape(1,512))
            X.append(phrases_emb_list)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
embed <tensorflow_hub.module.Module object at 0x7fb97bb1efd0>
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
# saving the encodings for future use
encodings = open("/content/drive/Shared drives/ChiragSaini SharedDrive/Precily Assignment/enc.pkl", "wb")
pickle.dump(X, encodings)
encodings.close()

In [0]:
# Converting the encoding into numpy arrays
text1 = np.array(X[0])
text2 = np.array(X[1])

In [8]:
text1.shape

(4023, 1, 512)

In [9]:
text2.shape

(4023, 1, 512)

In [0]:
#converting data to numpy aaray
X_data=np.array(X)

In [0]:
def get_cosine(vec1, vec2):
     
     numerator = sum(vec1 * vec2 )

     sum1 = sum([vec1[x]**2 for x in range(len(vec1))])
     sum2 = sum([vec2[x]**2 for x in range(len(vec2))])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

In [0]:
df["similarity_score"]=0

In [13]:
df.head()

,Unique_ID,text1,text2,similarity_score
0,0,savvy searchers fail to spot ads internet sear...,newcastle 2-1 bolton kieron dyer smashed home ...,0
1,1,millions to miss out on the net by 2025 40% o...,nasdaq planning $100m share sale the owner of ...,0
2,2,young debut cut short by ginepri fifteen-year-...,ruddock backs yapp s credentials wales coach m...,0
3,3,diageo to buy us wine firm diageo the world s...,mci shares climb on takeover bid shares in us ...,0
4,4,be careful how you code a new european directi...,media gadgets get moving pocket-sized devices ...,0


In [0]:
text2[0][0].shape

(512,)

In [0]:
values = []
for i in range(len(df)):
    cosine=get_cosine(text1[i][0], text2[i][0])
    values.append(cosine)
df["similarity_score"] = values

In [24]:
df.head()

,Unique_ID,text1,text2,similarity_score
0,0,savvy searchers fail to spot ads internet sear...,newcastle 2-1 bolton kieron dyer smashed home ...,0.166281
1,1,millions to miss out on the net by 2025 40% o...,nasdaq planning $100m share sale the owner of ...,0.640406
2,2,young debut cut short by ginepri fifteen-year-...,ruddock backs yapp s credentials wales coach m...,0.737810
3,3,diageo to buy us wine firm diageo the world s...,mci shares climb on takeover bid shares in us ...,0.693244
4,4,be careful how you code a new european directi...,media gadgets get moving pocket-sized devices ...,0.636458


In [0]:
#saving the pandas data frame to csv
df.to_csv('/content/drive/Shared drives/ChiragSaini SharedDrive/Precily Assignment/results.csv')

In [0]:
# Saving the data Frame as a pickle file
pickle_out = open("/content/drive/Shared drives/ChiragSaini SharedDrive/Precily Assignment/data.pkl","wb")
pickle.dump(df, pickle_out)
pickle_out.close()